In [1]:
import torch
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prolog

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch.optim as optim


def nb_errors(pred, truth):
    
    pred_class = pred.argmax(1)
    return (pred_class - truth != 0).sum().item()
        
def train_model(model, train_input, train_target, test_input, test_target,  epochs=500, batch_size=100, lr=0.1):
    

    
    optimizer = torch.optim.Adam(model.parameters())
    train_loss = []
    test_loss = []
    test_accuracy = []
    best_accuracy = 0
    best_epoch = 0
    
    for i in range(epochs):
        for b in range(0, train_input.size(0), batch_size):
            optimizer.zero_grad()
            output = model(train_input.narrow(0, b, batch_size))
            criterion = torch.nn.CrossEntropyLoss()
            loss = criterion(output, train_target.narrow(0, b, batch_size))
            loss.backward()
            optimizer.step()
        
        output_train = model(train_input)
        output_test = model(test_input)
        train_loss.append(criterion(output_train, train_target).item())
        test_loss.append(criterion(output_test, test_target).item())
        accuracy = 1 - nb_errors(output_test, test_target) / 1000
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_epoch = i+1
        test_accuracy.append(accuracy)
        
        if i%10 == 0:
            print('Epoch : ',i+1, '\t', 'test loss :', test_loss[-1], '\t', 'train loss', train_loss[-1])
        
    return train_loss, test_loss, test_accuracy, best_accuracy        

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        nb_hidden = 100
        self.conv1 = nn.Conv2d(2, 8, kernel_size=3)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)
        self.fc1 = nn.Linear(64, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
train_input, train_target, train_classes, test_input, test_target, test_classes = prolog.generate_pair_sets(1000)

In [5]:
model = ConvNet()

In [6]:
train_loss, test_loss, test_accuracy, best_accuracy = \
train_model(model, train_input, train_target, test_input, test_target, epochs=200, batch_size = 25)

Epoch :  1 	 test loss : 0.6751659512519836 	 train loss 0.6364557147026062
Epoch :  11 	 test loss : 0.7134395241737366 	 train loss 0.18941248953342438
Epoch :  21 	 test loss : 0.9486087560653687 	 train loss 0.018192851915955544
Epoch :  31 	 test loss : 1.2022900581359863 	 train loss 0.008553817868232727
Epoch :  41 	 test loss : 1.5456879138946533 	 train loss 0.046072859317064285
Epoch :  51 	 test loss : 1.8352928161621094 	 train loss 0.050852298736572266
Epoch :  61 	 test loss : 1.155489206314087 	 train loss 0.020578213036060333
Epoch :  71 	 test loss : 1.3722349405288696 	 train loss 0.002038997132331133
Epoch :  81 	 test loss : 1.520166039466858 	 train loss 0.0010207854211330414
Epoch :  91 	 test loss : 1.590381383895874 	 train loss 0.000893681775778532
Epoch :  101 	 test loss : 1.6455200910568237 	 train loss 0.0008310938137583435
Epoch :  111 	 test loss : 1.6919188499450684 	 train loss 0.0007935957401059568
Epoch :  121 	 test loss : 1.734903335571289 	 train l

In [8]:
best_accuracy

0.731